In [1]:
import numpy as np
import pandas as pd
import os 
import sys
import re 
import json
from tqdm import tqdm
import pandas as pd
import gensim
import gensim.parsing.preprocessing as gp
from gensim import utils
import re
import demoji
import unicodedata
import random

import sys
sys.path.append('/Users/HPhuc/AI_Project/')

from repos.utils import multiprocess

In [2]:
clean_data = pd.read_csv('/Users/HPhuc/Downloads/[ProductLabeled]MarketingContent_ver5_4.csv', index_col=0)

idc = random.randint(0, len(clean_data))
print(idc)
clean_data['content'][idc]

26425


'MỪNG TẾT LỚN CÙNG SAMSUNG Một Mùa Xuân hạnh phúc đang đến rất gần. Hãy để Samsung mang đến những sản phẩm Tủ Lạnh & Máy Giặt mới nhất trang bị cho tổ ấm của gia đình của bạn thêm sung túc cùng nhiều ưu đãi thật hấp dẫn. Mua Tủ Lạnh hoặc Máy Giặt Samsung từ 02/12/2019 đến hết 15/02/2020, bạn sẽ được tặng ngay những món quà Tết thật giá trị. Galaxy tab A+ S Pen 2019. Lò vi sóng. Máy ép trái cây. '

Define regex 

In [3]:
Hashtags_and_Mentions = "(@\S+|#\S+)"
Links                 = "https?://\S+|www\.\S+"
Emails                = '\S+@\S+'
Noise                 = "(\\W|\\d)"             # non-ascii and digits
Numbers               = '\d+'

**Prepare data**

In [4]:
raw_data = pd.read_csv('/Users/HPhuc/AI_Project/Ftech/[raw]marketing_content_final_1.csv')
raw_data

,content,label
0,"Với bạn, nhà là?\n\nLà nơi để trở về sau những...",LG_MarketingContent
1,"CHỦ THẺ ĐÓN LỢI THẾ, NHẬN ƯU ĐÃI TỚI 36%\n\nƯu...",LG_MarketingContent
2,"Độc đáo với công nghệ lọc 360°, Máy lọc không ...",LG_MarketingContent
3,"Chill nhạc chất, an toàn là nhất\n\nLG mong mu...",LG_MarketingContent
4,VÌ YÊU CON NÊN MẸ KHÓ?\nCỨ ĐỂ THINQ LO\n\nMuốn...,LG_MarketingContent
...,...,...
42544,🍭 Chào mừng bạn đến lớp học của CAKE!! Để xây ...,nguyenvt_raw
42545,🍩Mỹ phẩm đã sản xuất theo lít thì Cake cũng bo...,nguyenvt_raw
42546,Sống dễ (v) /ngồi nhà hưởng 5.2% lã...,nguyenvt_raw
42547,🌈Chúc mừng ngày Quốc Tế LGBT🌈 🍫Tha hồ chọn cho...,nguyenvt_raw


In [5]:
import random
idc = random.randint(0, len(raw_data))
print(idc)

sample = raw_data['content'].iloc[idc]
print(sample)


16744
Trang DXOMark - chuyên đánh giá Camera Phone đã tiến hành chụp và đánh giá chất lượng Camera của ZenFone 5, Với cảm biến hàng đầu là Sony IMX 363 Và với các thông số cụ thể như sau: Hai Camera sau: Camera chính: 12MP (f/1.8, 24mm, 1/2.55", 1.4µm, PDAF: lấy nét theo pha với công nghệ Dual-Pixel) Camera phụ: 8 MP (f/2.0, 12mm, 1/4", 1.12µm) Hai đèn LED Flash Camera trước: Một camera: 8 MP (f/2.0, 24mm, 1/4", 1.12µm), chống rung gyro EIS , khả năng quay phim FullHD 1080p Số điểm DXOMark Photo trên #ZenFone5 là 93 điểm - Một số điểm Xuất sắc Giá bán chính thức: 7,990,000 VND Bán độc quyền tại FPT shop Pre-order ngay: https://goo.gl/p9cQ8Y Tìm hiểu sản phẩm: https://goo.gl/X3wqN7 ============================================== #AIDualCameras - Camera chính: Sử dụng cảm biến hàng đầu Sony IMX 363, 12MP với Kích thước cảm biến lớn 1/2.55", kích thước điểm ảnh lên đến 1.4 Micron, Khẩu độ lớn F1.8, ISO 25, khả năng chống rung quang học OIS và EIS, tích hợp công nghệ lấy nét DualPixel PDAF.

**Analyse data**

In [6]:
raw_data['content'].apply(lambda s: str(re.findall(Links, s)))


0                               ['https://www.lg.com/vn/']
1        ['https://bit.ly/3vd8Y33', 'https://bit.ly/3vd...
2                              ['http://lge.ai/6177G6UjV']
3        ['https://www.lg.com/vn/tai-nghe-khong-day/lg-...
4        ['https://apple.co/3CP46lr', 'https://cutt.ly/...
                               ...                        
42544    ['https://cakevn.onelink.me/HHDV/CakeFB', 'htt...
42545    ['https://cakevn.onelink.me/HHDV/99611cd1', 'h...
42546    ['https://cakevn.onelink.me/HHDV/99611cd1', 'h...
42547    ['https://cakevn.onelink.me/HHDV/CakeFB', 'htt...
42548                                                   []
Name: content, Length: 42549, dtype: object

In [7]:
raw_data['content'].apply(lambda s: str(re.findall(Hashtags_and_Mentions, s)))


0                                           ['#LGVietnam']
1                                    ['#LG', '#LGVietnam']
2        ['#LG', '#LGVietnam', '#lifeisgood', '#LGPuriC...
3                            ['#LGVietnam', '#LGTONESL6S']
4           ['#LG', '#LGVietnam', '#ThinQ', '#LifeisGood']
                               ...                        
42544    ['#DễnhưănCake', '#DeNhuAnCake', '#NganhangsoC...
42545    ['#DễnhưănCake', '#DeNhuAnCake', '#NganhangsoC...
42546    ['#DễnhưănCake', '#DeNhuAnCake', '#Nganhan...
42547    ['#DễnhưănCake', '#DeNhuAnCake', '#NganhangsoC...
42548                                                   []
Name: content, Length: 42549, dtype: object

**Preprocessing**

Remove extra space

In [8]:
raw_data['clean_content'] = raw_data['content'].apply(lambda s: " ".join(s.split()))


print('content:', '\n', raw_data['content'].iloc[0])
print('clean_content:', '\n', raw_data['clean_content'].iloc[0])

content: 
 Với bạn, nhà là?

Là nơi để trở về sau những căng thẳng trong công việc và cuộc sống. Là nơi tràn ngập sự an lành và niềm vui bên những người thương yêu.

LG sẽ cùng bạn xây dựng và tô điểm cho từng giây phút đoàn tụ đều trở thành khoảnh khắc ấm cúng nhất qua những thiết bị gia dụng thông minh, tiện lợi và đi đầu trong mọi lĩnh vực.

LG - Vì cuộc sống tốt đẹp hơn: https://www.lg.com/vn/

#LGVietnam
clean_content: 
 Với bạn, nhà là? Là nơi để trở về sau những căng thẳng trong công việc và cuộc sống. Là nơi tràn ngập sự an lành và niềm vui bên những người thương yêu. LG sẽ cùng bạn xây dựng và tô điểm cho từng giây phút đoàn tụ đều trở thành khoảnh khắc ấm cúng nhất qua những thiết bị gia dụng thông minh, tiện lợi và đi đầu trong mọi lĩnh vực. LG - Vì cuộc sống tốt đẹp hơn: https://www.lg.com/vn/ #LGVietnam


Remove links

In [9]:
raw_data['clean_content'] = raw_data['clean_content'].apply(lambda s: re.sub(Links, " ", s))

print('content:', '\n', raw_data['content'].iloc[36573])
print('clean_content:', '\n', raw_data['clean_content'].iloc[36573])


content: 
 ♻️ ĐẬ𝐏 - ĐẢ𝐎 - 𝐍𝐇À𝐎 - 𝐍É𝐍 - 𝐂𝐇À - 𝐐𝐔𝐀𝐘 𝐊𝐇Ô𝐍𝐆 𝐓𝐑ƯỢ𝐓 𝐏𝐇Á𝐓 𝐍À𝐎 ♻️ Với máy giặt sở hữu CÔNG NGHỆ ĐỘC QUYỀN 6 MOTION, bạn sẽ tạm biệt chuỗi ngày phải gỡ áo quần rối xoắn, đồ lại sờn rách do chế độ giặt không phù hợp. Thay vào đó, bạn có thể tận hưởng áo quần thơm sạch, đẹp bền nhờ: ☑️ Trí thông minh nhân tạo giúp xác định chất liệu vải và trọng lượng để chọn ra chuyển động giặt tối ưu từ 20.000 thói quen giặt giũ. ☑️ Đặc biệt động cơ dẫn động trực tiếp 6 Motion tạo chuyển động mô phỏng cách giặt tay uyển chuyển giúp giặt kỹ áo quần mà không khiến chúng bị quấn vào nhau. 👉 𝐌𝐮𝐚 𝐧𝐠𝐚𝐲 𝐭ạ𝐢 >>> https://bit.ly/34Qi6w4 #dienmaycholon #maygiatLG #LG
clean_content: 
 ♻️ ĐẬ𝐏 - ĐẢ𝐎 - 𝐍𝐇À𝐎 - 𝐍É𝐍 - 𝐂𝐇À - 𝐐𝐔𝐀𝐘 𝐊𝐇Ô𝐍𝐆 𝐓𝐑ƯỢ𝐓 𝐏𝐇Á𝐓 𝐍À𝐎 ♻️ Với máy giặt sở hữu CÔNG NGHỆ ĐỘC QUYỀN 6 MOTION, bạn sẽ tạm biệt chuỗi ngày phải gỡ áo quần rối xoắn, đồ lại sờn rách do chế độ giặt không phù hợp. Thay vào đó, bạn có thể tận hưởng áo quần thơm sạch, đẹp bền nhờ: ☑️ Trí thông minh nhân tạo giúp xác định chất liệu

Remove email

In [10]:
# raw_data['clean_content'] = raw_data['content'].apply(lambda s: " ".join((re.sub(Emails, " ", s).split())))

# print('content:', '\n', raw_data['content'].iloc[4152])
# print('clean_content:', '\n', raw_data['clean_content'].iloc[4152])


Remove emojis

In [11]:
# raw_data['clean_content'] = raw_data['clean_content'].apply(lambda s: demoji.replace(s, ""))

# print('content:', raw_data['content'].iloc[30842])
# print('clean_content:', raw_data['clean_content'].iloc[30842])


Remove Hashtags and Mentions

In [12]:
raw_data['clean_content'] = raw_data['clean_content'].apply(lambda s: (re.sub(Hashtags_and_Mentions, " ", s)))


In [13]:
print(raw_data['content'].iloc[2], '\n')
print(raw_data['clean_content'].iloc[2])

Độc đáo với công nghệ lọc 360°, Máy lọc không khí LG PuriCare không chỉ giúp bạn lọc sạch hoàn toàn bụi khuẩn, mùi hôi mà còn lan tỏa khí sạch đến mọi ngóc ngách, dù bạn đặt máy ở bất kỳ vị trí nào trong căn phòng.

Khám phá chi tiết sản phẩm tại: http://lge.ai/6177G6UjV

#LG #LGVietnam #lifeisgood #LGPuriCareVietnam #PuriCare #ThanhLocKhongKhi #airpurify #protecthealth #healthcare #safeair #family #bestchoice #stayhome #staysafe #staystrong #covid #healthylife 

Độc đáo với công nghệ lọc 360°, Máy lọc không khí LG PuriCare không chỉ giúp bạn lọc sạch hoàn toàn bụi khuẩn, mùi hôi mà còn lan tỏa khí sạch đến mọi ngóc ngách, dù bạn đặt máy ở bất kỳ vị trí nào trong căn phòng. Khám phá chi tiết sản phẩm tại:                                    


Convert emotion and emticon to word

In [14]:
demoji.download_codes()

/Users/HPhuc/miniconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  """Entry point for launching an IPython kernel.


In [15]:
def emoji_to_words(idc):
    text = raw_data['clean_content'].iloc[idc]
    etow = [demoji.replace_with_desc(text, sep="__")]

    return etow

raw_data['clean_content'] = pd.DataFrame(multiprocess(emoji_to_words, range(raw_data.shape[0]), workers=6))


...multiprocessing...


100%|██████████| 42549/42549 [00:35<00:00, 1186.97it/s]


In [16]:
print(raw_data['content'].iloc[37784], '\n')
print(raw_data['clean_content'].iloc[37784])


𝐵𝐿𝐴𝐶𝐾 𝐹𝑅𝐼𝐷𝐴𝑌 𝐌Á𝐘 𝐆𝐈Ặ𝐓 𝐏𝐀𝐍𝐀𝐒𝐎𝐍𝐈𝐂 𝟗.𝟎 𝐊𝐆 𝐆𝐈Ả𝐌 𝐒Ố𝐂 𝟏.𝟑 𝐓𝐑𝐈Ệ𝐔 Giờ không mua thì đợi đến năm sau - ĐỪNG BỎ LỠ !! ⭐ Máy giặt Panasonic 9.0 Kg F90VS9 giảm 1 triệu ⏩ Giá Black Friday còn: 4.790.000đ (was: 5̶.̶9̶9̶0̶.̶0̶0̶0̶đ̶) - Giảm 1 Triệu Khi Mua Bộ Nồi Elmich - Nhập khẩu châu Âu - Tặng bộ PMH trị giá 700.000Đ MUA NGAY >> https://bit.ly/35LseFa ► Chỉ áp dụng đến 01.12.2019 trên toàn hệ thống Điện Máy Chợ Lớn ► Liên hệ ngay hotline: 028 3856 3388 để hỗ trợ đặt hàng và giao hàng tận nơi ----------- Nhận thêm ưu đãi đặc quyền: -  𝑻ặ𝒏𝒈 𝒏𝒈𝒂𝒚 𝒎á𝒚 𝒑𝒉𝒂 𝑪à 𝒑𝒉ê khi thanh toán bằng thẻ thanh toán quốc tế của ACB, VP Bank, BIDV, VIB, Viettinbank  trên máy POS đặt tại một trong các chi nhánh Điện Máy Chợ Lớn -  Đặ𝐜 𝐛𝐢ệ𝐭 𝐠𝐢ả𝐦 𝐭𝐡ê𝐦 𝟓𝟎𝟎.𝟎𝟎𝟎đ khi thanh toán bằng thẻ thanh toán quốc tế BIDV cho đơn hàng từ 7.000.000đ => Tham khảo thêm chi tiết ưu đãi tại >> https://bit.ly/35tcXZr #dienmaycholon #panasonic #maygiat 

𝐵𝐿𝐴𝐶𝐾 𝐹𝑅𝐼𝐷𝐴𝑌 𝐌Á𝐘 𝐆𝐈Ặ𝐓 𝐏𝐀𝐍𝐀𝐒𝐎𝐍𝐈𝐂 𝟗.𝟎 𝐊𝐆 𝐆𝐈Ả𝐌 𝐒Ố𝐂 𝟏.𝟑 𝐓𝐑𝐈Ệ𝐔 Giờ không mua thì đợi đến năm sau -

Replace number, date, mail by general token

In [17]:
raw_data['clean_content'] = raw_data['clean_content'].apply(lambda s: re.sub(Numbers, "<number>", s))


In [18]:
print(raw_data['content'].iloc[37784], '\n')
print(raw_data['clean_content'].iloc[37784])

𝐵𝐿𝐴𝐶𝐾 𝐹𝑅𝐼𝐷𝐴𝑌 𝐌Á𝐘 𝐆𝐈Ặ𝐓 𝐏𝐀𝐍𝐀𝐒𝐎𝐍𝐈𝐂 𝟗.𝟎 𝐊𝐆 𝐆𝐈Ả𝐌 𝐒Ố𝐂 𝟏.𝟑 𝐓𝐑𝐈Ệ𝐔 Giờ không mua thì đợi đến năm sau - ĐỪNG BỎ LỠ !! ⭐ Máy giặt Panasonic 9.0 Kg F90VS9 giảm 1 triệu ⏩ Giá Black Friday còn: 4.790.000đ (was: 5̶.̶9̶9̶0̶.̶0̶0̶0̶đ̶) - Giảm 1 Triệu Khi Mua Bộ Nồi Elmich - Nhập khẩu châu Âu - Tặng bộ PMH trị giá 700.000Đ MUA NGAY >> https://bit.ly/35LseFa ► Chỉ áp dụng đến 01.12.2019 trên toàn hệ thống Điện Máy Chợ Lớn ► Liên hệ ngay hotline: 028 3856 3388 để hỗ trợ đặt hàng và giao hàng tận nơi ----------- Nhận thêm ưu đãi đặc quyền: -  𝑻ặ𝒏𝒈 𝒏𝒈𝒂𝒚 𝒎á𝒚 𝒑𝒉𝒂 𝑪à 𝒑𝒉ê khi thanh toán bằng thẻ thanh toán quốc tế của ACB, VP Bank, BIDV, VIB, Viettinbank  trên máy POS đặt tại một trong các chi nhánh Điện Máy Chợ Lớn -  Đặ𝐜 𝐛𝐢ệ𝐭 𝐠𝐢ả𝐦 𝐭𝐡ê𝐦 𝟓𝟎𝟎.𝟎𝟎𝟎đ khi thanh toán bằng thẻ thanh toán quốc tế BIDV cho đơn hàng từ 7.000.000đ => Tham khảo thêm chi tiết ưu đãi tại >> https://bit.ly/35tcXZr #dienmaycholon #panasonic #maygiat 

𝐵𝐿𝐴𝐶𝐾 𝐹𝑅𝐼𝐷𝐴𝑌 𝐌Á𝐘 𝐆𝐈Ặ𝐓 𝐏𝐀𝐍𝐀𝐒𝐎𝐍𝐈𝐂 <number>.<number> 𝐊𝐆 𝐆𝐈Ả𝐌 𝐒Ố𝐂 <number>.<number> 𝐓𝐑𝐈Ệ𝐔 Giờ khô

Unicode Normalization

In [19]:
raw_data['clean_content'] = raw_data['clean_content'].apply(lambda s: unicodedata.normalize("NFKC", s))


In [20]:
print(raw_data['content'].iloc[37784], '\n')
print(raw_data['clean_content'].iloc[37784])

𝐵𝐿𝐴𝐶𝐾 𝐹𝑅𝐼𝐷𝐴𝑌 𝐌Á𝐘 𝐆𝐈Ặ𝐓 𝐏𝐀𝐍𝐀𝐒𝐎𝐍𝐈𝐂 𝟗.𝟎 𝐊𝐆 𝐆𝐈Ả𝐌 𝐒Ố𝐂 𝟏.𝟑 𝐓𝐑𝐈Ệ𝐔 Giờ không mua thì đợi đến năm sau - ĐỪNG BỎ LỠ !! ⭐ Máy giặt Panasonic 9.0 Kg F90VS9 giảm 1 triệu ⏩ Giá Black Friday còn: 4.790.000đ (was: 5̶.̶9̶9̶0̶.̶0̶0̶0̶đ̶) - Giảm 1 Triệu Khi Mua Bộ Nồi Elmich - Nhập khẩu châu Âu - Tặng bộ PMH trị giá 700.000Đ MUA NGAY >> https://bit.ly/35LseFa ► Chỉ áp dụng đến 01.12.2019 trên toàn hệ thống Điện Máy Chợ Lớn ► Liên hệ ngay hotline: 028 3856 3388 để hỗ trợ đặt hàng và giao hàng tận nơi ----------- Nhận thêm ưu đãi đặc quyền: -  𝑻ặ𝒏𝒈 𝒏𝒈𝒂𝒚 𝒎á𝒚 𝒑𝒉𝒂 𝑪à 𝒑𝒉ê khi thanh toán bằng thẻ thanh toán quốc tế của ACB, VP Bank, BIDV, VIB, Viettinbank  trên máy POS đặt tại một trong các chi nhánh Điện Máy Chợ Lớn -  Đặ𝐜 𝐛𝐢ệ𝐭 𝐠𝐢ả𝐦 𝐭𝐡ê𝐦 𝟓𝟎𝟎.𝟎𝟎𝟎đ khi thanh toán bằng thẻ thanh toán quốc tế BIDV cho đơn hàng từ 7.000.000đ => Tham khảo thêm chi tiết ưu đãi tại >> https://bit.ly/35tcXZr #dienmaycholon #panasonic #maygiat 

BLACK FRIDAY MÁY GIẶT PANASONIC <number>.<number> KG GIẢM SỐC <number>.<number> TRIỆU Giờ khô

Remove Noise

In [21]:
# non-ascii and digits
raw_data['clean_content'] = raw_data['clean_content'].apply(lambda s: (re.sub(Noise, " ", s)))


In [22]:
print(raw_data['content'].iloc[37784], '\n')
print(raw_data['clean_content'].iloc[37784])

𝐵𝐿𝐴𝐶𝐾 𝐹𝑅𝐼𝐷𝐴𝑌 𝐌Á𝐘 𝐆𝐈Ặ𝐓 𝐏𝐀𝐍𝐀𝐒𝐎𝐍𝐈𝐂 𝟗.𝟎 𝐊𝐆 𝐆𝐈Ả𝐌 𝐒Ố𝐂 𝟏.𝟑 𝐓𝐑𝐈Ệ𝐔 Giờ không mua thì đợi đến năm sau - ĐỪNG BỎ LỠ !! ⭐ Máy giặt Panasonic 9.0 Kg F90VS9 giảm 1 triệu ⏩ Giá Black Friday còn: 4.790.000đ (was: 5̶.̶9̶9̶0̶.̶0̶0̶0̶đ̶) - Giảm 1 Triệu Khi Mua Bộ Nồi Elmich - Nhập khẩu châu Âu - Tặng bộ PMH trị giá 700.000Đ MUA NGAY >> https://bit.ly/35LseFa ► Chỉ áp dụng đến 01.12.2019 trên toàn hệ thống Điện Máy Chợ Lớn ► Liên hệ ngay hotline: 028 3856 3388 để hỗ trợ đặt hàng và giao hàng tận nơi ----------- Nhận thêm ưu đãi đặc quyền: -  𝑻ặ𝒏𝒈 𝒏𝒈𝒂𝒚 𝒎á𝒚 𝒑𝒉𝒂 𝑪à 𝒑𝒉ê khi thanh toán bằng thẻ thanh toán quốc tế của ACB, VP Bank, BIDV, VIB, Viettinbank  trên máy POS đặt tại một trong các chi nhánh Điện Máy Chợ Lớn -  Đặ𝐜 𝐛𝐢ệ𝐭 𝐠𝐢ả𝐦 𝐭𝐡ê𝐦 𝟓𝟎𝟎.𝟎𝟎𝟎đ khi thanh toán bằng thẻ thanh toán quốc tế BIDV cho đơn hàng từ 7.000.000đ => Tham khảo thêm chi tiết ưu đãi tại >> https://bit.ly/35tcXZr #dienmaycholon #panasonic #maygiat 

BLACK FRIDAY MÁY GIẶT PANASONIC  number   number  KG GIẢM SỐC  number   number  TRIỆU Giờ khô

Word Segmentation

In [23]:
from underthesea import word_tokenize

def tokenize_word(idc):
    text = raw_data['clean_content'].iloc[idc]
    tokenized_word = [word_tokenize(text, format="text")]
    
    return tokenized_word

raw_data['clean_content'] = multiprocess(tokenize_word, range(raw_data.shape[0]), workers=7)


...multiprocessing...


100%|██████████| 42549/42549 [02:28<00:00, 286.59it/s] 


In [24]:
print(raw_data['content'].iloc[37784], '\n')
print(raw_data['clean_content'].iloc[37784])

𝐵𝐿𝐴𝐶𝐾 𝐹𝑅𝐼𝐷𝐴𝑌 𝐌Á𝐘 𝐆𝐈Ặ𝐓 𝐏𝐀𝐍𝐀𝐒𝐎𝐍𝐈𝐂 𝟗.𝟎 𝐊𝐆 𝐆𝐈Ả𝐌 𝐒Ố𝐂 𝟏.𝟑 𝐓𝐑𝐈Ệ𝐔 Giờ không mua thì đợi đến năm sau - ĐỪNG BỎ LỠ !! ⭐ Máy giặt Panasonic 9.0 Kg F90VS9 giảm 1 triệu ⏩ Giá Black Friday còn: 4.790.000đ (was: 5̶.̶9̶9̶0̶.̶0̶0̶0̶đ̶) - Giảm 1 Triệu Khi Mua Bộ Nồi Elmich - Nhập khẩu châu Âu - Tặng bộ PMH trị giá 700.000Đ MUA NGAY >> https://bit.ly/35LseFa ► Chỉ áp dụng đến 01.12.2019 trên toàn hệ thống Điện Máy Chợ Lớn ► Liên hệ ngay hotline: 028 3856 3388 để hỗ trợ đặt hàng và giao hàng tận nơi ----------- Nhận thêm ưu đãi đặc quyền: -  𝑻ặ𝒏𝒈 𝒏𝒈𝒂𝒚 𝒎á𝒚 𝒑𝒉𝒂 𝑪à 𝒑𝒉ê khi thanh toán bằng thẻ thanh toán quốc tế của ACB, VP Bank, BIDV, VIB, Viettinbank  trên máy POS đặt tại một trong các chi nhánh Điện Máy Chợ Lớn -  Đặ𝐜 𝐛𝐢ệ𝐭 𝐠𝐢ả𝐦 𝐭𝐡ê𝐦 𝟓𝟎𝟎.𝟎𝟎𝟎đ khi thanh toán bằng thẻ thanh toán quốc tế BIDV cho đơn hàng từ 7.000.000đ => Tham khảo thêm chi tiết ưu đãi tại >> https://bit.ly/35tcXZr #dienmaycholon #panasonic #maygiat 

['BLACK FRIDAY MÁY_GIẶT PANASONIC number number KG GIẢM SỐC number number TRIỆU Giờ không mua

In [25]:
raw_data

,content,label,clean_content
0,"Với bạn, nhà là?\n\nLà nơi để trở về sau những...",LG_MarketingContent,[Với bạn nhà là Là nơi để trở về sau những căn...
1,"CHỦ THẺ ĐÓN LỢI THẾ, NHẬN ƯU ĐÃI TỚI 36%\n\nƯu...",LG_MarketingContent,[CHỦ THẺ ĐÓN LỢI_THẾ_NHẬN ƯU_ĐÃI TỚI number Ưu...
2,"Độc đáo với công nghệ lọc 360°, Máy lọc không ...",LG_MarketingContent,[Độc_đáo với công_nghệ lọc number Máy lọc khôn...
3,"Chill nhạc chất, an toàn là nhất\n\nLG mong mu...",LG_MarketingContent,[Chill nhạc chất an_toàn là nhất LG mong_muốn ...
4,VÌ YÊU CON NÊN MẸ KHÓ?\nCỨ ĐỂ THINQ LO\n\nMuốn...,LG_MarketingContent,[VÌ YÊU CON NÊN MẸ KHÓ CỨ ĐỂ THINQ_LO Muốn chă...
...,...,...,...
42544,🍭 Chào mừng bạn đến lớp học của CAKE!! Để xây ...,nguyenvt_raw,[__lollipop__ Chào_mừng bạn đến lớp_học của CA...
42545,🍩Mỹ phẩm đã sản xuất theo lít thì Cake cũng bo...,nguyenvt_raw,[__doughnut__Mỹ phẩm đã sản_xuất theo lít thì ...
42546,Sống dễ (v) /ngồi nhà hưởng 5.2% lã...,nguyenvt_raw,[Sống dễ v ngồi nhà hưởng number number lãi_su...
42547,🌈Chúc mừng ngày Quốc Tế LGBT🌈 🍫Tha hồ chọn cho...,nguyenvt_raw,[__rainbow__Chúc mừng ngày Quốc_Tế LGBT__rainb...
